In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import time
import os


In [ ]:
# Check dataset
print(os.listdir("./dataset"))


In [ ]:

# Load data
data = pd.read_csv("./dataset/database.csv")
print("Original data shape:", data.shape)
print(data.head())

In [ ]:
# Select relevant columns
data = data[['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude']]
print("\nSelected columns:")
print(data.head())


In [ ]:

# Convert Date and Time to Timestamp
timestamp = []
for d, t in zip(data['Date'], data['Time']):
    try:
        ts = datetime.datetime.strptime(d+' '+t, '%m/%d/%Y %H:%M:%S')
        timestamp.append(time.mktime(ts.timetuple()))
    except ValueError:
        timestamp.append('ValueError')

timeStamp = pd.Series(timestamp)
data['Timestamp'] = timeStamp.values


In [ ]:


# Clean data - remove invalid timestamps
final_data = data.drop(['Date', 'Time'], axis=1)
final_data = final_data[final_data.Timestamp != 'ValueError']


In [ ]:


# Remove any remaining NaN values
final_data = final_data.dropna()

print("\nCleaned data shape:", final_data.shape)
print(final_data.head())
print("\nData types:")
print(final_data.dtypes)


In [ ]:

# ==================== VISUALIZATION ====================
try:
    from mpl_toolkits.basemap import Basemap
    
    longitudes = final_data["Longitude"].tolist()
    latitudes = final_data["Latitude"].tolist()
    
    fig = plt.figure(figsize=(12,10))
    plt.title("All Earthquake Affected Areas")
    
    m = Basemap(projection='mill',llcrnrlat=-80,urcrnrlat=80, 
                llcrnrlon=-180,urcrnrlon=180,lat_ts=20,resolution='c')
    
    x, y = m(longitudes, latitudes)
    m.plot(x, y, "o", markersize=2, color='blue', alpha=0.5)
    m.drawcoastlines()
    m.fillcontinents(color='coral',lake_color='aqua')
    m.drawmapboundary()
    m.drawcountries()
    plt.savefig('earthquake_map.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\nMap saved as 'earthquake_map.png'")
except ImportError:
    print("\nBasemap not available, skipping visualization")


In [ ]:

# ==================== PREPARE DATA ====================
X = final_data[['Timestamp', 'Latitude', 'Longitude']]
y = final_data[['Magnitude', 'Depth']]


In [ ]:


# Convert to numpy arrays and ensure float32 type
X = X.values.astype('float32')
y = y.values.astype('float32')

print("\nFeatures (X) shape:", X.shape)
print("Targets (y) shape:", y.shape)


In [ ]:

# Split data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("\nTrain set:", X_train.shape, y_train.shape)
print("Test set:", X_test.shape, y_test.shape)


In [ ]:

# Normalize features for better neural network performance
from sklearn.preprocessing import StandardScaler

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [ ]:


# ==================== RANDOM FOREST MODEL ====================
print("\n" + "="*50)
print("RANDOM FOREST REGRESSOR")
print("="*50)


In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Basic Random Forest
reg = RandomForestRegressor(random_state=42, n_estimators=100)
reg.fit(X_train, y_train)
y_pred_rf = reg.predict(X_test)

print("\nBasic Random Forest Results:")
print(f"R² Score: {reg.score(X_test, y_test):.4f}")
print(f"MSE: {mean_squared_error(y_test, y_pred_rf):.4f}")
print(f"MAE: {mean_absolute_error(y_test, y_pred_rf):.4f}")

In [ ]:

# GridSearch for Random Forest (optional - can take time)
print("\nPerforming GridSearch for Random Forest...")
from sklearn.model_selection import GridSearchCV

parameters = {'n_estimators': [50, 100, 200], 'max_depth': [10, 20, None]}
grid_obj = GridSearchCV(reg, parameters, cv=3, n_jobs=-1, verbose=1)
grid_fit = grid_obj.fit(X_train, y_train)
best_fit = grid_fit.best_estimator_

y_pred_rf_best = best_fit.predict(X_test)
print(f"\nBest Random Forest Parameters: {grid_fit.best_params_}")
print(f"Best R² Score: {best_fit.score(X_test, y_test):.4f}")


In [ ]:


# ==================== NEURAL NETWORK MODEL ====================
print("\n" + "="*50)
print("NEURAL NETWORK MODEL")
print("="*50)

In [ ]:

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Build Neural Network Model
def create_model(neurons=64, activation='relu', optimizer='adam', learning_rate=0.001):
    model = Sequential()
    model.add(Dense(neurons, activation=activation, input_shape=(3,)))
    model.add(Dropout(0.2))
    model.add(Dense(neurons//2, activation=activation))
    model.add(Dropout(0.2))
    model.add(Dense(neurons//4, activation=activation))
    model.add(Dense(2))  # Output layer for regression (Magnitude, Depth)
    
    if optimizer == 'adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = keras.optimizers.SGD(learning_rate=learning_rate)
    else:
        opt = optimizer
    
    model.compile(optimizer=opt, loss='mse', metrics=['mae'])
    
    return model

In [ ]:


# Create and train model
model = create_model(neurons=128, activation='relu', optimizer='adam', learning_rate=0.001)

print("\nModel Architecture:")
model.summary()


In [ ]:


# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)


In [ ]:

# Train model
print("\nTraining Neural Network...")
history = model.fit(
    X_train_scaled, y_train_scaled,
    batch_size=32,
    epochs=100,
    verbose=1,
    validation_data=(X_test_scaled, y_test_scaled),
    callbacks=[early_stopping, reduce_lr]
)

In [ ]:

# Evaluate model
print("\n" + "="*50)
print("NEURAL NETWORK EVALUATION")
print("="*50)

[test_loss, test_mae] = model.evaluate(X_test_scaled, y_test_scaled, verbose=0)
print(f"\nTest Loss (MSE): {test_loss:.4f}")
print(f"Test MAE: {test_mae:.4f}")

In [ ]:

# Make predictions
y_pred_nn_scaled = model.predict(X_test_scaled)
y_pred_nn = scaler_y.inverse_transform(y_pred_nn_scaled)

In [ ]:



# Calculate metrics
mse_nn = mean_squared_error(y_test, y_pred_nn)
mae_nn = mean_absolute_error(y_test, y_pred_nn)
r2_nn = r2_score(y_test, y_pred_nn)

print(f"\nNeural Network Results (on original scale):")
print(f"MSE: {mse_nn:.4f}")
print(f"MAE: {mae_nn:.4f}")
print(f"R² Score: {r2_nn:.4f}")

In [ ]:


# ==================== PLOT TRAINING HISTORY ====================
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

axes[0].plot(history.history['loss'], label='Training Loss')
axes[0].plot(history.history['val_loss'], label='Validation Loss')
axes[0].set_title('Model Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss (MSE)')
axes[0].legend()
axes[0].grid(True)

axes[1].plot(history.history['mae'], label='Training MAE')
axes[1].plot(history.history['val_mae'], label='Validation MAE')
axes[1].set_title('Model MAE')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('MAE')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()
print("\nTraining history saved as 'training_history.png'")

In [ ]:

# ==================== PREDICTION COMPARISON ====================
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Magnitude prediction
axes[0].scatter(y_test[:, 0], y_pred_nn[:, 0], alpha=0.5)
axes[0].plot([y_test[:, 0].min(), y_test[:, 0].max()], 
             [y_test[:, 0].min(), y_test[:, 0].max()], 'r--', lw=2)
axes[0].set_xlabel('Actual Magnitude')
axes[0].set_ylabel('Predicted Magnitude')
axes[0].set_title('Magnitude Prediction')
axes[0].grid(True)

# Depth prediction
axes[1].scatter(y_test[:, 1], y_pred_nn[:, 1], alpha=0.5)
axes[1].plot([y_test[:, 1].min(), y_test[:, 1].max()], 
             [y_test[:, 1].min(), y_test[:, 1].max()], 'r--', lw=2)
axes[1].set_xlabel('Actual Depth')
axes[1].set_ylabel('Predicted Depth')
axes[1].set_title('Depth Prediction')
axes[1].grid(True)

plt.tight_layout()
plt.savefig('prediction_comparison.png', dpi=300, bbox_inches='tight')
plt.show()
print("Prediction comparison saved as 'prediction_comparison.png'")


In [ ]:


# ==================== SAVE MODELS ====================
# Save Keras model
model.save('earthquake_model.h5')
print("\nNeural Network model saved as 'earthquake_model.h5'")

# Save scalers
import pickle
with open('scaler_X.pkl', 'wb') as f:
    pickle.dump(scaler_X, f)
with open('scaler_y.pkl', 'wb') as f:
    pickle.dump(scaler_y, f)
print("Scalers saved as 'scaler_X.pkl' and 'scaler_y.pkl'")

# Save Random Forest model
with open('random_forest_model.pkl', 'wb') as f:
    pickle.dump(best_fit, f)
print("Random Forest model saved as 'random_forest_model.pkl'")


In [ ]:


# ==================== PREDICTION FUNCTION ====================
def predict_earthquake(timestamp, latitude, longitude):
    """
    Predict earthquake magnitude and depth
    
    Parameters:
    - timestamp: Unix timestamp
    - latitude: Latitude value
    - longitude: Longitude value
    
    Returns:
    - magnitude: Predicted magnitude
    - depth: Predicted depth in km
    """
    # Prepare input
    X_input = np.array([[timestamp, latitude, longitude]], dtype='float32')
    
    # Scale input
    X_input_scaled = scaler_X.transform(X_input)
    
    # Predict
    y_pred_scaled = model.predict(X_input_scaled, verbose=0)
    y_pred = scaler_y.inverse_transform(y_pred_scaled)
    
    magnitude = y_pred[0][0]
    depth = y_pred[0][1]
    
    return magnitude, depth


In [ ]:

# ==================== TEST PREDICTION ====================
print("\n" + "="*50)
print("EXAMPLE PREDICTION")
print("="*50)


In [ ]:

# Use a sample from test set
sample_idx = 0
sample_timestamp = X_test[sample_idx, 0]
sample_lat = X_test[sample_idx, 1]
sample_lon = X_test[sample_idx, 2]

pred_mag, pred_depth = predict_earthquake(sample_timestamp, sample_lat, sample_lon)

print(f"\nInput:")
print(f"  Timestamp: {sample_timestamp}")
print(f"  Latitude: {sample_lat:.4f}")
print(f"  Longitude: {sample_lon:.4f}")
print(f"\nPrediction:")
print(f"  Magnitude: {pred_mag:.2f}")
print(f"  Depth: {pred_depth:.2f} km")
print(f"\nActual:")
print(f"  Magnitude: {y_test[sample_idx, 0]:.2f}")
print(f"  Depth: {y_test[sample_idx, 1]:.2f} km")

print("\n" + "="*50)
print("EARTHQUAKE PREDICTION MODEL COMPLETE!")
print("="*50)